In [ ]:
# Part of the VexIR2Vec Project, under the AGPL V3.0 License. See the
# LICENSE file or <https://www.gnu.org/licenses/> for license information.

import heapq
import sys, re
import numpy as np
import pandas as pd 
from collections import OrderedDict
from scipy import spatial

In [ ]:
def findVec(str1, src):
    f = open(src)
    start = content.upper().find("\n"+str1.upper()+":[") + len(str1) + 3
    if(start == -1):
        print(str1, " not found")
        return 
    else:
        end = content.find("]", start)
        vecstr = content[start:end].split(", ")
        vec = [float(element) for element in vecstr]
        # print(vec)
        return vec

In [ ]:
def genSimilarityTable(vec, src):
    lines = [line.strip('\n\t') for line in open(src)]
    cosineDict = {}
    euclDict = {}
    for line in lines:
        opcode = line[0:line.find(":[")].upper()
        valueStr = line[line.find(":[")+2:-2].split(", ")
        value = [float(element) for element in valueStr]
        cosineDict[opcode] = spatial.distance.cosine(vec, value)
        euclDict[opcode] = spatial.distance.euclidean(vec, value)
    return cosineDict, euclDict

In [ ]:
def findTopk(dict1, k, values):
    sortedByVal = dict(sorted(dict1.items(), key=lambda x: x[1]))
    del sortedByVal[values[0].upper()]
    del sortedByVal[values[1].upper()]
    del sortedByVal[values[2].upper()]
    return {k: sortedByVal[k] for k in list(sortedByVal)[:k]}

In [ ]:
refFile = '/path/to/analogies.txt'
analogies = [line.strip('\n') for line in open(refFile)]

totalCnt = 0
fileCorrectCnt = {}

fns = [
        "/path/to/seedEmbeddings.txt"
        ] 

for fileName in fns:
    print('\n', '\033[93m' + '\033[4m' + fileName + '\033[0m')
    avg = []
    correctCnt = 0
    for analogy in analogies:
        totalCnt = totalCnt+1
        values = analogy.split(" ")
    
        vecA = findVec(values[0], fileName)
        vecB = findVec(values[1], fileName)
        vecC = findVec(values[2], fileName)
        vecD = np.asarray(vecB) - np.asarray(vecA) + np.asarray(vecC)
        
        
        # print(vecD)
        cosineDict, euclDict = genSimilarityTable(vecD, fileName)
        topKCosineDict = findTopk(cosineDict,5,values)
        if values[3].upper() in topKCosineDict:
            correctCnt = correctCnt+1
            print(values, ' : ', '\033[92m' + u'\u2713' + '\033[0m', topKCosineDict.keys())
            avg.append(topKCosineDict[values[3].upper()])
        else:
            print(values, ' : ', '\033[91m' + u'\u00D7' + '\033[0m', topKCosineDict.keys())
    fileCorrectCnt[fileName] = correctCnt
print("Total #analogies = ", totalCnt/len(fns))
print(fileCorrectCnt)   